In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from rasa_sdk import Action
from rasa_sdk.events import SlotSet, AllSlotsReset, Restarted
import zomatopy
import json
import re
import pandas as pd
from pandas.io.json import json_normalize
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Defining Global variables:
# List of tier 1 and tier 2 cities
tier1_2_cities = ['agra', 'ahmedabad', 'ajmer', 'aligarh', 'allahabad', 'amravati', 'amritsar',
                  'asansol', 'aurangabad', 'bangalore', 'bareilly', 'belgaum', 'bhavnagar',
                  'bhiwadi', 'bhopal', 'bhubaneshwar', 'bijapur', 'bikaner', 'bokaro', 'chandigarh',
                  'chennai', 'coimbatore', 'cuttack', 'dehradun', 'dhanbad', 'dharwad', 'durg bhilai',
                  'durgapur', 'erode', 'faridabad', 'firozabad', 'ghaziabad', 'goa', 'gorakhpur',
                  'gulbarga', 'guntur', 'gurgaon', 'guwahati', 'gwalior', 'hyderabad', 'indore',
                  'jabalpur', 'jaipur', 'jalandhar', 'jammu', 'jamnagar', 'jamshedpur', 'jhansi',
                  'jodhpur', 'kakinada', 'kannur', 'kanpur', 'kochi', 'kolhapur', 'kolkata', 'kollam',
                  'kota', 'kottayam', 'kozhikode', 'kurnool', 'lucknow', 'ludhiana', 'madurai',
                  'malappuram', 'mangalore', 'mathura', 'meerut', 'moradabad', 'mumbai',
                  'mysore', 'nagpur', 'nanded', 'nashik', 'nellore', 'new delhi',
                  'noida', 'palakkad', 'patna', 'pune', 'raipur', 'rajahmundry', 'rajkot',
                  'ranchi', 'rourkela', 'salem', 'sangli', 'siliguri', 'solapur', 'srinagar',
                  'sultanpur', 'surat', 'thrissur', 'tirunelveli', 'tiruppur', 'trichy',
                  'trivandrum', 'ujjain', 'vadodara', 'varanasi', 'vijayawada', 'visakhapatnam','warangal']

In [3]:
# List of restaurants from the action_restaurant_search.
# The list was originally populated in action_restaurant_search as well as action_send_mail. 
# However, the action_send_mail was taking longer than 10sec many times thus resulting in a error due to rasa configuration setting.
# Thus, retrieving the list once and storing it globally for acces in action_send_mail due reduce the processing there. 
email_list = ""

In [4]:
def restaurant_details(city,cuisines,upper_budget=5000,lower_budget=50):
    '''
    This function will take the values of location, cuisine and budget range, 
    and will return a DataFrame Object that contains top 10 restaurants in 
    Descending order of rating in the location. 
    
    Parameters:
    location: Location sprecifid by the user.
    cuisines : Cuisine Specified by User.
    upper_budget: Maximum cost for two people, default value is 5000, if none specified.
    lower_budget: Minimum cost for two people, default value is 50, if none specified.
    '''
    #config={ "user_key":"f4924dc9ad672ee8c4f8c84743301af5"}
    config={ "user_key":"2cd53b995b0818e559df301ec3b98a06"}
    zomato = zomatopy.initialize_app(config)
    loc = city#tracker.get_slot('location')
    cuisine = cuisines#tracker.get_slot('cuisine')
    location_detail=zomato.get_location(loc, 1)
    d1 = json.loads(location_detail)
    lat=d1["location_suggestions"][0]["latitude"]
    lon=d1["location_suggestions"][0]["longitude"]
    cuisines_dict={'bakery':5,'chinese':25,'cafe':30,'italian':55,'biryani':7,'north indian':50,'south indian':85}
    results=zomato.restaurant_search("", lat, lon, str(cuisines_dict.get(cuisine)), 100)
    d = json.loads(results)
    df = json_normalize(d['restaurants'])
    rest= df[['restaurant.user_rating.aggregate_rating', 'restaurant.name','restaurant.location.city','restaurant.location.address','restaurant.average_cost_for_two']]
    rest.sort_values(['restaurant.user_rating.aggregate_rating', 'restaurant.average_cost_for_two'],ascending=False,inplace=True)
    rest['restaurant.location.city'] = rest['restaurant.location.city'].str.lower().str.strip()
    rest['restaurant.location.address'] = rest['restaurant.location.address'].str.strip()
    rest=rest[(rest['restaurant.average_cost_for_two']<=upper_budget) & (rest['restaurant.average_cost_for_two']>=lower_budget)& (rest['restaurant.location.city']== loc)]
    #rest=rest[(rest['restaurant.average_cost_for_two']<=upper_budget) & (rest['restaurant.average_cost_for_two']>=lower_budget)]

    rest= rest.astype(str)
    rest.reset_index(inplace=True)
    rest.drop(columns=['index'],inplace=True)
    return rest.head(10)

In [5]:
def get_restaurant_chat_details(rest):
    '''
    This function takes a DataFrame object (rest) which contains restaurant details
    and return a String that will be used to display restaurants found on the chatbot window.
    '''
    restaurants = rest.head(5)
    details = 'Found top restaurant with following details: \n'
    for i in restaurants.index:
        details = details+  str(restaurants.index.get_loc(i)+1)+'. '+restaurants['restaurant.name'][i]+' in '+restaurants['restaurant.location.address'][i] + ' has been rated '+restaurants['restaurant.user_rating.aggregate_rating'][i]+ '\n'
    return details

In [7]:
def get_list_for_email(rest):
    '''
    This function will take a data frame object containing the restaurant details 
    and will return a string in html tagged format, which will be used to send email to the user. 
    '''
    email_listitem=''
    for i in rest.index:
        email_listitem=email_listitem+ '<li><b>'+ rest['restaurant.name'][i]+'</b><br/>User Rating: '+rest['restaurant.user_rating.aggregate_rating'][i]+'<br/>Location: '+ rest['restaurant.location.address'][i]+'</b><br/>Cost for two: '+rest['restaurant.average_cost_for_two'][i]+'</li><br/>'
    return email_listitem

In [16]:
city = 'bombay'
cuisine = 'South Indian'

In [17]:
respdf= restaurant_details(city= city,cuisines=cuisine)

In [21]:
respdf['restaurant.name'].count()

0

In [23]:
if respdf['restaurant.name'].count() > 0:
    email_list = get_list_for_email(respdf)
    response = get_restaurant_chat_details(respdf)
    dispatcher.utter_message(response)
    return [SlotSet('noresults',"false")]
else:
    return [SlotSet('noresults',"true")]

SyntaxError: 'return' outside function (<ipython-input-23-e03863ef4d03>, line 5)

In [24]:
class ActionSearchRestaurants(Action):
    def name(self):
        return 'action_search_restaurants'

    def run(self, dispatcher, tracker, domain):
        config={ "user_key":"f4924dc9ad672ee8c4f8c84743301af5"}
        zomato = zomatopy.initialize_app(config)

        city = tracker.get_slot('city')
        cuisine = tracker.get_slot('cuisine')
        budget = tracker.get_slot('budget')

        global email_list 
        email_list = ""

        if budget == 'low':
            upper_budget = 299
            lower_budget=10
        elif budget == 'medium':
            upper_budget = 700
            lower_budget=301
        elif budget == 'high':
            upper_budget = 10000
            lower_budget=701
        else:
            upper_budget = 10000
            lower_budget=10

        respdf= restaurant_details(city= city,cuisines=cuisine,upper_budget=upper_budget,lower_budget=lower_budget)
        
        if respdf['restaurant.name'].count() > 0:
            email_list = get_list_for_email(respdf)
            response = get_restaurant_chat_details(respdf)
            dispatcher.utter_message(response)
            return [SlotSet('noresults',"false")]
        else:
            dispatcher.utter_message("Oops! Could not find results in that area!")
            return [SlotSet('noresults',"true")]

In [27]:
 respdf= restaurant_details(city='raipur',cuisines='North Indian')
        

In [28]:
respdf

,restaurant.user_rating.aggregate_rating,restaurant.name,restaurant.location.city,restaurant.location.address,restaurant.average_cost_for_two
0,4.7,Gadh Kalewa,raipur,"Great Eastern Rd, Near Mahant Ghasidas Samarak...",400
1,4.7,Axis Juice,raipur,"B- 16, Sector 1, Devendra Nagar, Raipur",300
2,4.6,KFC,raipur,"Shop 34, Ground Floor, Magnato Mall Dharsiwa, ...",450
3,4.6,Manju Mamta Restaurant,raipur,"Shardha Chowk, MG Road, Jawahar Nagar, Raipur",350
4,4.4,McDonald's,raipur,"Ground Floor, Magneto Mall, Shankar Nagar, Raipur",500
5,4.4,Rolls Mania,raipur,"47, Ground Floor, Crystal Arcade, Shankar Naga...",250
6,4.2,Ashoka Biryani,raipur,"Near Amanaka Bridge, Mahoba Bazaar, GE Road, S...",650
7,4.2,Anna Punjabi,raipur,"Pachpedi Naka Chowk, Beside Nest, Mahaveer Nag...",500
8,4.1,Pizza Hut,raipur,"Jail Road, Raipur, Devendra Nagar, Raipur",600
9,4.1,Kebab Junction,raipur,Sultan tower arya samaj mandir road baijnath p...,400


In [31]:
if respdf['restaurant.name'].count() > 0:
    print('1')
            #email_list = get_list_for_email(respdf)
            #response = get_restaurant_chat_details(respdf)
            #dispatcher.utter_message(response)
            #return [SlotSet('noresults',"false")]
else:
    print('0')
            #dispatcher.utter_message("Oops! Could not find results in that area!")
            #return [SlotSet('noresults',"true")]

1


In [26]:
class ActionCheckEmail(Action):
    def name(self):
        return 'action_check_email'

    def run(self, dispatcher, tracker, domain):
        emailid = tracker.get_slot('email')
        # Make a regular expression 
        # for validating an Email 
        regex = '^\w+([\.-]?\w+)*@\w+([\.-]?\w+)*(\.\w{2,3})+$'

        
        try:
            if(re.search(regex,emailid)):
                return [SlotSet('email_validate',"true")]
            else:
                return [SlotSet('email_validate',"false")]
        except:
            return [SlotSet('email_validate',"false")]            